# PriMAT: Multi-animal tracking

In this notebook, we want to demonstrate how to train a tracking model from a few hundred frames. We will train a model that is able to track lemurs and feeding boxes with labelled images which can be downloaded [here](https://owncloud.gwdg.de/index.php/s/Mq4m9k1B74cN6ys) (-> Training Images / LemurBoxSep22 ).

In [5]:
!nvidia-smi

Sat Aug 24 13:29:41 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.1.0+cu118.html
!pip install yacs==0.1.8
!pip install opencv-python
!pip install progress==1.6
!pip install scikit-learn==1.2.2

Looking in links: https://data.pyg.org/whl/torch-2.1.0+cu118.html
  Preparing metadata (setup.py) ... done
  Created wheel for progress: filename=progress-1.6-py3-none-any.whl size=9613 sha256=d1e1b2e7d400352c9889bd1a41dc43410620531057aa591f06d62c11ffb3815d
  Stored in directory: /root/.cache/pip/wheels/a2/68/5f/c339b20a41659d856c93ccdce6a33095493eb82c3964aac5a1
Successfully built progress
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 95.7 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.2
    Uninstalling scikit-learn-1.3.2:
      Successfully uninstalled scikit-learn-1.3.2


In [11]:
!git clone https://github.com/ecker-lab/PriMAT-tracking.git
!cd PriMAT-tracking/
!mkdir data
!mkdir models
!mkdir exp
!mkdir videos

mkdir: cannot create directory ‘data’: File exists
mkdir: cannot create directory ‘models’: File exists
mkdir: cannot create directory ‘exp’: File exists


## Training

### Extract data

You have to upload the folder with the data to the colab space on the left. I uploaded it as a tar file and extracted it into the folder "data". The only important thing is that after this step you have training material in the folder data.

In [12]:
%cd data
!tar -xvf ../../LemurBoxSep22.tar >/dev/null!tar
%cd ..

/content/PriMAT-tracking/data
/content/PriMAT-tracking


### Run training script

In the following cell you start the training. Before this you have to make sure a few things:
- The data_cfg file has to point to the correct location of your data. Open it (it is in src/lib/cfg/....json) and adapt the path (for me it is /content/PriMAT-tracking/ in colab).
- Give your experiment a name (--exp_id) so that you can find the model afterwards in exp/mot/exp_id/.
-

In [18]:
%%shell
cd src
python train.py mot --exp_id colab_test\
                    --load_model ''\
                    --num_epochs 10\
                    --lr_step 5\
                    --lr '1e-4'\
                    --data_cfg '../src/lib/cfg/lemur_box.json'\
                    --store_opt\
                    --arch hrnet_32\
                    --gpus 0\
                    --batch_size 2\
                    --seed 13\
                    --reid_cls_names lemur,box\
                    --val_intervals 10\
                    --save_all
cd ..

/content/PriMAT-tracking/src/lib
training chunk_sizes: [2]
The output will be saved to  /content/PriMAT-tracking/src/lib/../../exp/mot/colab_test
Setting up data...
dataset summary
OrderedDict([('lemur_box_train', defaultdict(<class 'int'>, {0: 1361.0, 1: 1343.0}))])
start index
OrderedDict()
heads {'hm': 2, 'wh': 2, 'id': 128, 'reg': 2}
Namespace(task='mot', dataset='jde', exp_id='colab_test', test=False, load_model='', load_tracking_model='', load_id_model='', resume=False, store_opt=True, gpus=[0], num_workers=8, not_cuda_benchmark=False, seed=13, print_iter=0, hide_data_time=False, metric='loss', vis_thresh=0.5, arch='hrnet_32', head_conv=256, down_ratio=4, reid_dim=128, no_aug_hsv=True, squared_bboxes=False, move_px=0, zoom_min=1, zoom_max=1, gc_dim=128, input_h=608, input_w=1088, ltrb=False, lr=0.0001, lr_step=[5], num_epochs=10, batch_size=2, master_batch_size=2, num_iters=-1, val_intervals=10, trainval=False, save_all=True, K=50, not_prefetch_test=False, line_thickness=1, id_in

Your models will be in exp/mot/exp_id and end with .pth.

## Inference

### Apply to videos

If you want to have video output, you will need to activate ffmpeg.

In [20]:
!apt-get update && apt-get install -y ffmpeg

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [945 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Ign:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:13 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu 

- If you want the model you just trained, you can directly change the path to ../exp/mot/exp_id/model_last.pth (or any other model you want). We use the pretrained lemur model from [here](https://owncloud.gwdg.de/index.php/s/Mq4m9k1B74cN6ys) (-> Models).
- I uploaded one validation video (Eval8.mp4) into data/Videos.
- If you set output_format to video, a video will be saved to output_root/output_name. If you set it to text, only the tracking output as a .txt file will be saved.

In [31]:
!pip install motmetrics==1.2.0
!pip install lap==0.4.0
!pip install cython_bbox==0.1.3

In [34]:
%%shell

cd src

python demo.py mot  --load_tracking_model ../models/lemur_tracking.pth\
                    --conf_thres 0.02\
                    --det_thres 0.5\
                    --new_overlap_thres 0.8\
                    --sim_thres 0.8\
                    --input_video ../data/Videos/Eval8.mp4\
                    --output_root ../videos/test/\
                    --output_name test_video\
                    --store_opt\
                    --line_thickness 2\
                    --debug_info\
                    --arch hrnet_32\
                    --output_format video\
                    --reid_cls_names "lemur,box"\
                    --proportion_iou 0.2\
                    --double_kalman


cd ..

/content/PriMAT-tracking/src/lib
2024-08-24 14:55:03 [DEBUG]: matplotlib data path: /usr/local/lib/python3.10/dist-packages/matplotlib/mpl-data
2024-08-24 14:55:03 [DEBUG]: CONFIGDIR=/root/.config/matplotlib
2024-08-24 14:55:03 [DEBUG]: interactive is False
2024-08-24 14:55:03 [DEBUG]: platform is linux
2024-08-24 14:55:03 [DEBUG]: CACHEDIR=/root/.cache/matplotlib
2024-08-24 14:55:03 [DEBUG]: Using fontManager instance from /root/.cache/matplotlib/fontlist-v330.json
2024-08-24 14:55:03 [INFO]: NumExpr defaulting to 2 threads.
training chunk_sizes: [12]
The output will be saved to  /content/PriMAT-tracking/src/lib/../../exp/mot/default
Net input image size: 1088×608
heads {'hm': 2, 'wh': 2, 'id': 128, 'reg': 2}
2024-08-24 14:55:04 [INFO]: Starting tracking...
Length of the video: 346 frames
Creating model...
2024-08-24 14:55:04 [INFO]: => init weights from normal distribution
2024-08-24 14:55:06 [INFO]: Processing frame 0 (100000.00 fps)
2024-08-24 14:55:42 [INFO]: Processing frame 200 